In [1]:
import numpy as np
import pandas as pd

import requests
import json
import time
from bs4 import BeautifulSoup
from lxml import etree
from bs4 import BeautifulSoup
from pandas import DataFrame
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [2]:
cities_1k_df = pd.read_csv(r'E:\Report\战略\半室外需求验证\源数据\citi\us-cities-top-1k.csv')
cities_1k_df['cities_states'] = cities_1k_df['City']+','+cities_1k_df['State']
cities_df = cities_1k_df.copy().sort_values(by=['Population'])

cities_df

,City,State,Population,lat,lon,cities_states
961,Panama City,Florida,36877,30.158813,-85.660206,"Panama City,Florida"
552,Beloit,Wisconsin,36888,42.508348,-89.031777,"Beloit,Wisconsin"
434,Spanish Fork,Utah,36956,40.114955,-111.654923,"Spanish Fork,Utah"
722,Keizer,Oregon,37064,44.990119,-123.026208,"Keizer,Oregon"
346,Weslaco,Texas,37093,26.159519,-97.990837,"Weslaco,Texas"
...,...,...,...,...,...,...
53,Philadelphia,Pennsylvania,1553165,39.952584,-75.165222,"Philadelphia,Pennsylvania"
646,Houston,Texas,2195914,29.760427,-95.369803,"Houston,Texas"
594,Chicago,Illinois,2718782,41.878114,-87.629798,"Chicago,Illinois"
953,Los Angeles,California,3884307,34.052234,-118.243685,"Los Angeles,California"


# 爬数据

In [76]:
#Define Search
search_location = cities_df['cities_states'].drop_duplicates(keep = 'first').tolist()
search_location.remove('Lima,Ohio')
#search_location = cities_df['State'].drop_duplicates().tolist()
search_list = ['restaurant, curbside pickup']
search_limit = 50
offset_limit = 50

In [77]:
len(search_location)

999

In [61]:
#定义搜索公式
def search_restaurant(set_location,offset_number,search_term):
    # 根据输入的地点和循环次数抓取该地点的所有餐厅数据

    url = 'https://api.yelp.com/v3/businesses/search' #Yelp Fusion
    headers = {'Authorization': 'Bearer {}'.format('_p2c3uy9Fsct8mWbxBtAdHjgZ8pLm6K151jeo0DawEHB24aSvsS3w3aQc7lopX29z_cENlennOnhbGsiYuJQQBgSijB7NOY_L1KFoyx3pkMrYbLBUNn6Z1W3ynBeYnYx')} #个人密钥
    url_params = { #parameters passed to the API
        'term': search_term,
        'location': set_location,
        'offset': offset_number,
        'limit': search_limit # Maximum return of results per request (ref: API documentation).
    }

    restaurant_response = requests.get(url, headers=headers, params=url_params).json()
    restaurant_output = pd.DataFrame.from_dict(restaurant_response['businesses'])
    return restaurant_output # Returns a df

In [64]:
search_restaurant(search_location[37],0,'restaurant')

TypeError: 'NoneType' object is not subscriptable

In [44]:
#定义城市循环公式
def loop_city(start_city_number,target_city_number, offset_number, search_term):
    loop_city_merge = pd.DataFrame()
    for i in np.arange(start_city_number,target_city_number,1) :
        loop_city_append = search_restaurant(search_location[i], offset_number, search_term)
        loop_city_merge = DataFrame(loop_city_merge.append(loop_city_append, ignore_index=True))
    return loop_city_merge

In [32]:
#定义同城循环公式
def loop_restaurant(target_city_number, search_term):
    loop_restaurant_merge = pd.DataFrame()
    for n in np.arange(0, offset_limit, 50):
        loop_restaurant_append = loop_city(target_city_number,n, search_term)
        loop_restaurant_merge = DataFrame(loop_restaurant_merge.append(loop_restaurant_append, ignore_index=True))
    return loop_restaurant_merge

In [33]:
#定义种类循环公式
def loop_search(target_city_number):
    loop_search_merge = pd.DataFrame()
    for m in np.arange(0, len(search_list), 1):
        loop_search_append = loop_restaurant(target_city_number,search_list[m])
        loop_search_merge = DataFrame(loop_search_merge.append(loop_search_append, ignore_index=True))
    return loop_search_merge

In [118]:
data_curbside_100 = loop_city(0,100,50,'restaurant, curbside pickup')

data_curbside_100

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,RoTdVh4JUEB-Z0iBvV--Rw,sonic-drive-in-duarte,Sonic Drive-In,https://s3-media2.fl.yelpcdn.com/bphoto/V2nCtT...,False,https://www.yelp.com/biz/sonic-drive-in-duarte...,1075,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",2.5,"{'latitude': 34.1392483, 'longitude': -117.982...","[delivery, pickup]",$,"{'address1': '1070 E Huntington Dr', 'address2...",+16262567902,(626) 256-7902,2381.012694
1,hQ5WKHrtP91nSbl6FqWNwA,marketplace-cafe-arcadia-6,Marketplace Cafe,https://s3-media3.fl.yelpcdn.com/bphoto/e6GziR...,False,https://www.yelp.com/biz/marketplace-cafe-arca...,216,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 34.1370999, 'longitude': -118.0517}",[],$$,"{'address1': '400 S Baldwin Ave', 'address2': ...",+16268216363,(626) 821-6363,4155.085813
2,ECHSC2v7ASIRIviywoPZoQ,islands-restaurant-pasadena,Islands Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/4p942r...,False,https://www.yelp.com/biz/islands-restaurant-pa...,523,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",3.5,"{'latitude': 34.150750964126, 'longitude': -11...",[delivery],$$,"{'address1': '3533 E Foothill Blvd', 'address2...",+16263516543,(626) 351-6543,6454.828147
3,_1sx2durs4tnc6X0xBSCmg,hook-burger-pasadena,Hook Burger,https://s3-media3.fl.yelpcdn.com/bphoto/BNTb_h...,False,https://www.yelp.com/biz/hook-burger-pasadena?...,794,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.0,"{'latitude': 34.1503542184429, 'longitude': -1...",[],$$,"{'address1': '3453 E Foothill Blvd', 'address2...",+16263510011,(626) 351-0011,6660.206561
4,LijLC5WgQtZc5xvZtc8PZw,the-boiling-crab-rosemead,The Boiling Crab,https://s3-media3.fl.yelpcdn.com/bphoto/ZUGE__...,False,https://www.yelp.com/biz/the-boiling-crab-rose...,651,"[{'alias': 'cajun', 'title': 'Cajun/Creole'}, ...",3.5,"{'latitude': 34.073011, 'longitude': -118.07092}",[],$$,"{'address1': '3514 Rosemead Blvd', 'address2':...",+16264073773,(626) 407-3773,9729.619500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1547,I5zWMBrBKutosdevR2Mrzg,anjappar-chettinad-indian-restaurant-santa-clara,Anjappar Chettinad Indian Restaurant,https://s3-media4.fl.yelpcdn.com/bphoto/xje-sc...,False,https://www.yelp.com/biz/anjappar-chettinad-in...,458,"[{'alias': 'indpak', 'title': 'Indian'}]",3.0,"{'latitude': 37.336335188543565, 'longitude': ...","[pickup, delivery]",$$,"{'address1': '777 Lawerence Expy', 'address2':...",+14082435500,(408) 243-5500,7231.944971
1548,Lr6wRaymLWzEcs0e1d00kw,pizza-hut-san-jose-17,Pizza Hut,https://s3-media4.fl.yelpcdn.com/bphoto/Zp_XA5...,False,https://www.yelp.com/biz/pizza-hut-san-jose-17...,30,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2.0,"{'latitude': 37.3131118, 'longitude': -121.878...",[delivery],$,"{'address1': '226 W Alma Ave', 'address2': 'St...",+14082885400,(408) 288-5400,7967.984277
1549,BY5bHRfeRXGZM8cZaceE6w,go-fish-poke-bar-san-jose,Go Fish Poke Bar,https://s3-media4.fl.yelpcdn.com/bphoto/osMftR...,False,https://www.yelp.com/biz/go-fish-poke-bar-san-...,1009,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.0,"{'latitude': 37.303987, 'longitude': -122.032909}","[pickup, delivery]",$$,"{'address1': '1183 S De Anza Blvd', 'address2'...",+14083430988,(408) 343-0988,7266.888060
1550,cCwwJX1tLE-AP8TnB-uXMw,scotts-chowder-house-san-jose,Scott's Chowder House,https://s3-media2.fl.yelpcdn.com/bphoto/9wo6Uf...,False,https://www.yelp.com/biz/scotts-chowder-house-...,55,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",3.5,"{'latitude': 37.33294, 'longitude': -121.88773}","[pickup, delivery]",$$,"{'address1': '200 S First St', 'address2': 'St...",+14085316421,(408) 531-6421,8595.759812


In [120]:
data_curbside_200 = loop_city(100,200,50,'restaurant, curbside pickup')

data_curbside_200

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,3BDGMMROL5w3UgI1jYfNKw,my-pie-pizza-holladay,My Pie Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/H-nQIH...,False,https://www.yelp.com/biz/my-pie-pizza-holladay...,113,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.6694729, 'longitude': -111.824...","[pickup, delivery]","{'address1': '4655 S 2300th E', 'address2': 'S...",+18017485888,(801) 748-5888,20926.408959,NaN
1,yE5x8rtZG6FgA8crAzml9g,even-stevens-sandwiches-draper,Even Stevens Sandwiches,https://s3-media3.fl.yelpcdn.com/bphoto/bkU83u...,False,https://www.yelp.com/biz/even-stevens-sandwich...,314,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.0,"{'latitude': 40.5267185, 'longitude': -111.875...",[],"{'address1': '541 E 12300th S', 'address2': 'S...",+18019800755,(801) 980-0755,7961.751253,$
2,Wh-HiKe1abx9GBk1i41ZLQ,costa-vida-riverton,Costa Vida,https://s3-media1.fl.yelpcdn.com/bphoto/tCto-Y...,False,https://www.yelp.com/biz/costa-vida-riverton?a...,55,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",2.5,"{'latitude': 40.50733, 'longitude': -111.97731}",[],"{'address1': '3627 W 13400 S', 'address2': '',...",+18014468000,(801) 446-8000,1262.016763,$
3,9klOzbjMLcr7PmL0U9JfRQ,texas-roadhouse-sandy-2,Texas Roadhouse,https://s3-media3.fl.yelpcdn.com/bphoto/U7d-_D...,False,https://www.yelp.com/biz/texas-roadhouse-sandy...,360,"[{'alias': 'steak', 'title': 'Steakhouses'}, {...",3.5,"{'latitude': 40.56, 'longitude': -111.896}",[],"{'address1': '200 W 10600th S', 'address2': ''...",+18015727718,(801) 572-7718,7827.579181,$$
4,W9IlPLShN9BbQqQwuPShtg,cafe-rio-mexican-grill-herriman,Cafe Rio Mexican Grill,https://s3-media2.fl.yelpcdn.com/bphoto/jKtjSx...,False,https://www.yelp.com/biz/cafe-rio-mexican-gril...,190,"[{'alias': 'mexican', 'title': 'Mexican'}]",2.0,"{'latitude': 40.50902940563278, 'longitude': -...",[delivery],"{'address1': '5506 W 13400th S', 'address2': '...",+18019884800,(801) 988-4800,4547.951700,$
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1247,Fm3MHvc8-4Y08iULSRCzdA,marios-italian-cafe-palm-desert-6,Mario's Italian Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/5cRpjR...,False,https://www.yelp.com/biz/marios-italian-cafe-p...,145,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.5,"{'latitude': 33.762017, 'longitude': -116.300337}",[delivery],"{'address1': '78152 Varner Rd', 'address2': 'S...",+17603458900,(760) 345-8900,24727.773915,$$
1248,98mJXR72Xiw3hVmKoL30yA,wincos-pizza-indio,Winco's Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/A8B9AQ...,False,https://www.yelp.com/biz/wincos-pizza-indio?ad...,13,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 33.73941, 'longitude': -116.21378}",[],"{'address1': '42600 Jackson St', 'address2': '...",+17603424138,(760) 342-4138,33143.332281,$
1249,KZhY60NHHhxADfzwCxXcKQ,blaze-pizza-palm-desert,Blaze Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/99RBSK...,False,https://www.yelp.com/biz/blaze-pizza-palm-dese...,348,"[{'alias': 'salad', 'title': 'Salad'}, {'alias...",3.5,"{'latitude': 33.7211451, 'longitude': -116.384...","[delivery, pickup]","{'address1': '73393 California 111', 'address2...",+17608945030,(760) 894-5030,19734.705767,$
1250,0KY--250_qwpICabh4eq9w,applebees-grill-bar-beaumont,Applebee's Grill + Bar,https://s3-media3.fl.yelpcdn.com/bphoto/1vkGcp...,False,https://www.yelp.com/biz/applebees-grill-bar-b...,190,"[{'alias': 'tradamerican', 'title': 'American ...",3.0,"{'latitude': 33.92885, 'longitude': -116.950568}","[delivery, pickup]","{'address1': '1585 E 6th St', 'address2': '', ...",+19518453638,(951) 845-3638,38124.782749,$$


In [ ]:
data_curbside_300 = loop_city(200,300,50,'restaurant, curbside pickup')

data_curbside_300

In [ ]:
data_curbside_400 = loop_city(300,400,50,'restaurant, curbside pickup')

data_curbside_400

In [ ]:
data_curbside_500 = loop_city(400,500,50,'restaurant, curbside pickup')

data_curbside_500

In [ ]:
data_curbside_600 = loop_city(500,600,50,'restaurant, curbside pickup')

data_curbside_600

In [ ]:
data_curbside_700 = loop_city(600,700,50,'restaurant, curbside pickup')

data_curbside_700

In [ ]:
data_curbside_800 = loop_city(700,800,50,'restaurant, curbside pickup')

data_curbside_800

In [ ]:
data_curbside_900 = loop_city(800,900,50,'restaurant, curbside pickup')

data_curbside_900

In [ ]:
data_curbside_1000 = loop_city(900,999,50,'restaurant, curbside pickup')

data_curbside_1000

In [ ]:
data_2 = pd.concat([data_curbside_100,data_curbside_200,data_curbside_300,data_curbside_400,data_curbside_500,data_curbside_600,data_curbside_700,data_curbside_800,data_curbside_900,data_curbside_1000])
data_2 = data_2.drop_duplicates('id',keep = 'first')

data_2

In [116]:
data_1

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,uehJbEEu68Dey9u_eNfzhQ,the-grand-marlin-panama-city-beach-panama-city,The Grand Marlin - Panama City Beach,https://s3-media2.fl.yelpcdn.com/bphoto/hTomT9...,False,https://www.yelp.com/biz/the-grand-marlin-pana...,489,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"{'latitude': 30.147905, 'longitude': -85.7451131}",[delivery],$$$,"{'address1': '5323 N Lagoon Dr', 'address2': N...",+18502491500,(850) 249-1500,6707.857119
1,-sG599tgqFft9c6HD5GBdA,los-antojitos-mexican-restaurant-panama-city-3,Los Antojitos Mexican Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/vs1v6q...,False,https://www.yelp.com/biz/los-antojitos-mexican...,243,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",3.5,"{'latitude': 30.171392, 'longitude': -85.700935}","[delivery, pickup]",$$,"{'address1': '1236 Beck Ave', 'address2': '', ...",+18507846633,(850) 784-6633,2070.049207
2,X1kJsqwepcLbFNc_37jDfA,the-place-downtown-panama-city,The Place Downtown,https://s3-media2.fl.yelpcdn.com/bphoto/pjeG98...,False,https://www.yelp.com/biz/the-place-downtown-pa...,38,"[{'alias': 'pubs', 'title': 'Pubs'}, {'alias':...",4.0,"{'latitude': 30.155765, 'longitude': -85.66069}",[delivery],$$,"{'address1': '429 Harrison Ave', 'address2': N...",+18508180590,(850) 818-0590,2327.145375
3,xgBcBl3nkuZyzyqgqkE59Q,capt-andersons-restaurant-and-waterfront-marke...,Capt Anderson's Restaurant & Waterfront Market,https://s3-media1.fl.yelpcdn.com/bphoto/fIecQG...,False,https://www.yelp.com/biz/capt-andersons-restau...,479,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",3.5,"{'latitude': 30.1497025659363, 'longitude': -8...",[delivery],$$,"{'address1': '5551 N Lagoon Dr', 'address2': '...",+18502342225,(850) 234-2225,6807.715968
4,AubNMgqOZ-SVsFTp8V69_Q,pizza-hut-panama-city-4,Pizza Hut,https://s3-media3.fl.yelpcdn.com/bphoto/0Nzc3s...,False,https://www.yelp.com/biz/pizza-hut-panama-city...,15,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",1.5,"{'latitude': 30.1892681609801, 'longitude': -8...",[delivery],$,"{'address1': '221 W 23rd St', 'address2': '', ...",+18507692219,(850) 769-2219,2750.111143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4821,HdYmIA2jRBbKnU1mGJ9a3Q,the-quarter-brooklyn-brooklyn,The Quarter Brooklyn,https://s3-media2.fl.yelpcdn.com/bphoto/mWt9Co...,False,https://www.yelp.com/biz/the-quarter-brooklyn-...,222,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 40.6874192552749, 'longitude': -7...","[pickup, delivery]",$$,"{'address1': '87 Lafayette Ave', 'address2': '...",+13479874012,(347) 987-4012,2588.250776
4823,xTnivXEdEtXrnhp-z4JGvg,la-parisienne-new-york-5,La Parisienne,https://s3-media2.fl.yelpcdn.com/bphoto/ywbWsB...,False,https://www.yelp.com/biz/la-parisienne-new-yor...,682,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",4.5,"{'latitude': 40.70961, 'longitude': -74.0093799}","[pickup, delivery]",$$,"{'address1': '9 Maiden Ln', 'address2': None, ...",+16467564911,(646) 756-4911,1350.108963
4829,vbGqzhH_exG5HOG1I23kvg,park-side-restaurant-corona-2,Park Side Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/Qj2HV9...,False,https://www.yelp.com/biz/park-side-restaurant-...,809,"[{'alias': 'italian', 'title': 'Italian'}]",4.0,"{'latitude': 40.743843, 'longitude': -73.856118}",[delivery],$$$,"{'address1': '107-01 Corona Ave', 'address2': ...",+17182719871,(718) 271-9871,12402.075037
4832,_-EQR2vNYg-b1eZwAgp6JQ,tanoreen-brooklyn-2,Tanoreen,https://s3-media2.fl.yelpcdn.com/bphoto/bpdxzU...,False,https://www.yelp.com/biz/tanoreen-brooklyn-2?a...,945,"[{'alias': 'mideastern', 'title': 'Middle East...",4.0,"{'latitude': 40.6308, 'longitude': -74.02775}","[pickup, delivery]",$$,"{'address1': '7523 3rd Ave', 'address2': '', '...",+17187485600,(718) 748-5600,8768.541359


In [ ]:
data_2

In [117]:
data_1.to_csv('E:\Report\战略\半室外需求验证\city_curbside_citi_1.csv')

# 确认是否有户外就餐

In [ ]:
url_list = data['url'].to_list()

In [ ]:
#定义attribute关键词确认公式
def search_detail(url,detail_key):
    yelp_r = requests.get(url)
    yelp_soup = BeautifulSoup(yelp_r.content, 'html.parser')
    yelp_text = str(yelp_soup)
    if detail_key in yelp_text:
        return 1
    else:
        return 0

In [ ]:
#定义循环搜索公式
def confirmation_result(detail_key):
    temp = 0
    total = len(url_list)
    confirmation_list = []
    for link in np.arange(0, len(url_list), 1):
        confirmation = search_detail(url_list[link],detail_key)
        confirmation_list.append(confirmation)
        temp += 1
        print('\r' + '[Progress]:[%s%s]%.2f%%;' % ('█' * int(temp*20/total), ' ' * (20-int(temp*20/total)),float(temp/total*100)), end='')
        time.sleep(0.01)
    return confirmation_list

In [ ]:
No_Outdoor_Seating= confirmation_result('No Outdoor Seating')

In [ ]:
data['No Outdoor Seating'] = No_Outdoor_Seating
data